In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.autograd as autograd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_min, x_max, y_min, y_max, t_min, t_max = -100, 100, -100, 100, 0, 10


In [ ]:
# ---------------------------------------------
# PINN model definition
# ---------------------------------------------
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        # Simple fully-connected neural network:
        #   input : (x, y, t)  →  3-dimensional
        #   hidden: three layers of 128 neurons with Tanh activation
        #   output: scalar u(x,y,t) ∈ (0,1) via Sigmoid
        #
        # Sigmoid is used here because the solution u represents a
        # transported "blob" with values between 0 and 1 (like a concentration).
        self.net = nn.Sequential(
            nn.Linear(3, 128), nn.Tanh(),
            nn.Linear(128, 128), nn.Tanh(),
            nn.Linear(128, 128), nn.Tanh(),
            nn.Linear(128, 1), nn.Sigmoid()
        )

    def forward(self, x):
        # Forward pass: simply apply the Sequential network
        # x shape: (N, 3) where columns are [x, y, t]
        # returns: (N, 1) predicted scalar field u(x,y,t)
        return self.net(x)


# ---------------------------------------------
# Velocity field a(x,y), b(x,y)
# ---------------------------------------------
def velocity(x, y):
    """
    Compute the radial velocity field (a, b) given spatial coordinates (x, y).

    The analytical definition is:
        a(x,y) = x / sqrt(x^2 + y^2)
        b(x,y) = y / sqrt(x^2 + y^2)

    which corresponds to a unit vector pointing radially outward.

    At the origin (x,y) = (0,0), r = 0 and the direction is undefined,
    so we manually set the velocity to (0,0) there to avoid division by zero.
    """
    # Radial distance r = sqrt(x^2 + y^2)
    r = torch.sqrt(x**2 + y**2)

    # a = x / r, but only where r > 0. At r == 0, set a = 0.
    a = torch.where(r > 0, x / r, torch.zeros_like(r))

    # b = y / r, but only where r > 0. At r == 0, set b = 0.
    b = torch.where(r > 0, y / r, torch.zeros_like(r))

    # a, b shapes: same as x,y (usually (N,1))
    return a, b


# ---------------------------------------------
# PDE residual loss: advection–diffusion equation
# ---------------------------------------------
def pde_loss(model, xyt):
    """
    Compute the physics (PDE) loss for the advection–diffusion equation:

        u_t + a(x,y) u_x + b(x,y) u_y = u_xx + u_yy

    This is enforced at randomly sampled collocation points (x, y, t).

    Args:
        model: the PINN model u(x,y,t)
        xyt  : tensor of shape (N, 3) with columns [x, y, t]

    Returns:
        Scalar tensor: mean squared PDE residual over all collocation points.
    """
    # We need gradients w.r.t. xyt to compute u_x, u_y, u_t, etc.
    xyt.requires_grad_(True)

    # Forward pass: u(x,y,t)
    u = model(xyt)  # shape (N,1)

    # First-order derivatives of u w.r.t. (x,y,t)
    # autograd.grad returns a tensor of shape (N,3):
    #   grads[:,0] = ∂u/∂x, grads[:,1] = ∂u/∂y, grads[:,2] = ∂u/∂t
    grads = autograd.grad(
        outputs=u,
        inputs=xyt,
        grad_outputs=torch.ones_like(u),
        create_graph=True)[0]

    # Split the gradient into components, each of shape (N,1)
    u_x, u_y, u_t = grads[:, 0:1], grads[:, 1:2], grads[:, 2:3]

    # Second derivatives u_xx and u_yy via another autograd call
    # First: differentiate u_x w.r.t xyt, then select x-component for u_xx
    u_xx = autograd.grad(
        outputs=u_x,
        inputs=xyt,
        grad_outputs=torch.ones_like(u_x),
        create_graph=True)[0][:, 0:1]

    # Similarly: differentiate u_y w.r.t xyt, then select y-component for u_yy
    u_yy = autograd.grad(
        outputs=u_y,
        inputs=xyt,
        grad_outputs=torch.ones_like(u_y),
        create_graph=True
    )[0][:, 1:2]

    # Extract spatial coordinates (x,y) from xyt
    x, y = xyt[:, 0:1], xyt[:, 1:2]

    # Compute radial velocity components a(x,y), b(x,y)
    a, b = velocity(x, y)  # shapes (N,1)

    # PDE residual for each point:
    #   r(x,y,t) = u_t + a u_x + b u_y - u_xx - u_yy
    # We reshape a,b to (N,1) just to be explicit.
    residual = u_t + a.view(-1, 1) * u_x + b.view(-1, 1) * u_y - u_xx - u_yy

    # Physics loss = mean squared residual
    return (residual**2).mean()


# ---------------------------------------------
# Initial condition loss
# ---------------------------------------------
def ic_loss(model, xyt_ic, u_ic):
    """
    Initial condition loss at t = 0.

    Args:
        model : PINN model u(x,y,t)
        xyt_ic: tensor of shape (N_ic, 3) with t=0 for all rows
        u_ic  : tensor of shape (N_ic, 1) with prescribed IC values,
                e.g. 1 inside the disk and 0 outside.

    Returns:
        Mean squared error between predicted and prescribed IC.
    """
    u_pred = model(xyt_ic)
    return ((u_pred - u_ic) ** 2).mean()


# ---------------------------------------------
# Neumann boundary condition loss
# ---------------------------------------------
def bc_loss(model, xyt_bc):
    """
    Neumann boundary condition loss: ∇u · n = 0 on all edges of the domain.

    The outward normal n is:
      - left  boundary (x = x_min): n = (-1, 0)
      - right boundary (x = x_max): n = ( 1, 0)
      - bottom boundary (y = y_min): n = (0, -1)
      - top    boundary (y = y_max): n = (0,  1)

    For each boundary point, we compute ∇u and its dot product with n,
    then minimize (∇u · n)^2.

    Args:
        model : PINN model u(x,y,t)
        xyt_bc: tensor of shape (N_bc, 3) containing boundary points.

    Returns:
        Mean squared normal-derivative (Neumann) residual.
    """
    # Need gradients w.r.t. xyt_bc for computing spatial derivatives
    xyt_bc.requires_grad_(True)

    # Forward pass on boundary points
    u = model(xyt_bc)  # (N_bc,1)

    # Gradient of u w.r.t. (x,y,t)
    grads = autograd.grad(
        outputs=u,
        inputs=xyt_bc,
        grad_outputs=torch.ones_like(u),
        create_graph=True
    )[0]  # shape (N_bc, 3)

    # Extract x,y coordinates (1D tensors: shape (N_bc,))
    x, y = xyt_bc[:, 0], xyt_bc[:, 1]

    # Initialize normal components nx, ny as zeros
    # These will be filled with ±1 depending on which boundary each point lies on.
    nx = torch.zeros_like(x)
    ny = torch.zeros_like(y)

    # Left boundary: x == x_min → normal (-1, 0)
    nx[x == x_min] = -1
    # Right boundary: x == x_max → normal (+1, 0)
    nx[x == x_max] = 1

    # Bottom boundary: y == y_min → normal (0, -1)
    ny[y == y_min] = -1
    # Top boundary: y == y_max → normal (0, +1)
    ny[y == y_max] = 1

    # Normal derivative ∂u/∂n = ∇u · n = u_x * nx + u_y * ny
    grad_n = grads[:, 0] * nx + grads[:, 1] * ny  # shape (N_bc,)

    # We want ∂u/∂n = 0, so penalize (∂u/∂n)^2
    return (grad_n.view(-1, 1) ** 2).mean()


# ---------------------------------------------
# Sampling functions for collocation, IC, and BC
# ---------------------------------------------
def sample_domain(N):
    """
    Sample N random collocation points inside the full space-time domain:

        x ∈ [x_min, x_max],
        y ∈ [y_min, y_max],
        t ∈ [t_min, t_max].

    Returns:
        Tensor of shape (N, 3) with columns [x, y, t].
    """
    x = np.random.uniform(x_min, x_max, N)
    y = np.random.uniform(y_min, y_max, N)
    t = np.random.uniform(t_min, t_max, N)

    # np.c_[x, y, t] concatenates the 1D arrays column-wise to shape (N,3)
    return torch.tensor(np.c_[x, y, t], dtype=torch.float32).to(device)


def sample_ic(N):
    """
    Sample N points for the initial condition at t = 0.

    Initial condition:
        u(x,y,0) = 1   if x^2 + y^2 < (12.5)^2  (inside the circle)
                  = 0   otherwise (outside the circle)

    Returns:
        xyt_ic : tensor (N,3) with t=0
        u_ic  : tensor (N,1) with corresponding IC values (0 or 1)
    """
    # Random spatial positions in the domain
    x = np.random.uniform(x_min, x_max, N)
    y = np.random.uniform(y_min, y_max, N)

    # Time is fixed to 0 for all IC points
    t = np.zeros(N)

    # Indicator for being inside the initial circle of radius 12.5
    # u = 1 inside, 0 outside
    u = ((x**2 + y**2) < (12.5)**2).astype(np.float32)

    xyt_ic = torch.tensor(np.c_[x, y, t], dtype=torch.float32).to(device)
    u_ic = torch.tensor(u[:, None], dtype=torch.float32).to(device)
    return xyt_ic, u_ic


def sample_bc(N):
    """
    Sample boundary points on all four edges of the spatial domain
    for Neumann boundary conditions.

    For each edge we create N points:
        - y = y_min, x ∈ [x_min, x_max]
        - y = y_max, x ∈ [x_min, x_max]
        - x = x_min, y ∈ [y_min, y_max]
        - x = x_max, y ∈ [y_min, y_max]

    For each boundary point we also randomly select a time t ∈ [t_min, t_max].

    Returns:
        Tensor of shape (4N, 3) with all boundary points stacked.
    """
    pts = []

    # We encode each boundary by a tuple:
    #   (range_for_free_coord, fixed_value, axis_flag)
    #
    # axis_flag = 1  → vary x, fix y (horizontal edges)
    # axis_flag = 0  → vary y, fix x (vertical edges)
    for v, const, axis in [
        ((x_min, x_max), y_min, 1),  # bottom edge: y = y_min
        ((x_min, x_max), y_max, 1),  # top edge:    y = y_max
        ((y_min, y_max), x_min, 0),  # left edge:   x = x_min
        ((y_min, y_max), x_max, 0),  # right edge:  x = x_max
    ]:
        # v is a 2-tuple (start, end); linspace gives N points along that edge
        val = np.linspace(*v, N)
        consts = np.full(N, const)

        # Random times for each boundary point
        t = np.random.uniform(t_min, t_max, N)

        # For horizontal edges (axis=1): x varies, y is constant
        # For vertical edges   (axis=0): y varies, x is constant
        if axis == 0:
            # Vertical edge: x is constant, val is y
            pts.append(np.c_[consts, val, t])
        else:
            # Horizontal edge: y is constant, val is x
            pts.append(np.c_[val, consts, t])

    # Stack all four edge point sets into a single array of shape (4N,3)
    return torch.tensor(np.vstack(pts), dtype=torch.float32).to(device)


# ---------------------------------------------
# Training loop (Adam optimizer)
# ---------------------------------------------
model = PINN().to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10000):
    # Sample new collocation points for PDE, IC, and BC at each epoch.
    xyt_f = sample_domain(2000)        # PDE collocation points
    xyt_ic, u_ic = sample_ic(1000)     # IC points and labels
    xyt_bc = sample_bc(500)            # BC points

    # Compute three loss terms
    lf = pde_loss(model, xyt_f)               # physics (PDE) loss
    li = ic_loss(model, xyt_ic, u_ic)         # initial condition loss
    lb = bc_loss(model, xyt_bc)               # boundary condition loss

    # Total loss with weights: IC and BC are scaled up to enforce them strongly
    loss = lf + 1000 * li + 100 * lb

    # Standard PyTorch training step
    opt.zero_grad()
    loss.backward()
    opt.step()

    # Print progress every 500 epochs
    if epoch % 500 == 0:
        print(
            f"Epoch {epoch}: "
            f"Total={loss.item():.4f} "
            f"PDE={lf.item():.4f} "
            f"IC={li.item():.4f} "
            f"BC={lb.item():.4f}"
        )


# ---------------------------------------------
# Generate predictions on a regular (x,y) grid
# and make an animation over time
# ---------------------------------------------

# 2D spatial grid for visualization: 101×101 points
xg = np.linspace(x_min, x_max, 101)
yg = np.linspace(y_min, y_max, 101)
X, Y = np.meshgrid(xg, yg)

frames = []  # list to store predicted u(x,y,t) for each time snapshot

# Evaluate the model at 50 time instants between t=0 and t=10
for tval in np.linspace(0, 10, 50):
    # Build (x,y,t) for all grid points at fixed time tval:
    #   - X.ravel(), Y.ravel() → flatten to 1D arrays
    #   - np.full_like(X.ravel(), tval) → same length, constant tval
    xyt = np.stack(
        [X.ravel(), Y.ravel(), np.full_like(X.ravel(), tval)],
        axis=1
    )  # shape (101*101, 3)

    xyt_tensor = torch.tensor(xyt, dtype=torch.float32).to(device)

    # Inference without gradients
    with torch.no_grad():
        # Predict u on the grid and reshape back to (101, 101)
        u_pred = model(xyt_ten


In [ ]:
### "Finite Element Method"
# NOTE: This is actually an explicit finite-difference scheme for the
#       2D advection–diffusion equation on a structured grid.

# -----------------------------
# Domain and discretization setup
# -----------------------------

L = 200          # Physical side length of the square domain in both x and y
Nx = 201         # Number of grid points in x-direction
Ny = 201         # Number of grid points in y-direction
Nt = 200         # Number of time steps
T = 10.0         # Final simulation time

# Spatial and temporal step sizes
dx = L / (Nx - 1)    # Grid spacing in x (domain length / number of intervals)
dy = L / (Ny - 1)    # Grid spacing in y
dt = T / Nt          # Time step size

# 1D coordinate arrays for x and y
x = np.linspace(-100, 100, Nx)  # x ∈ [-100, 100]
y = np.linspace(-100, 100, Ny)  # y ∈ [-100, 100]

# 2D meshgrid: X[i,j], Y[i,j] give the physical coordinates of node (i,j)
X, Y = np.meshgrid(x, y)

# -----------------------------
# Velocity field (radial)
# -----------------------------

# Radial distance from origin at each grid point
R = np.sqrt(X**2 + Y**2)

# a(x,y) = x / r , b(x,y) = y / r, but avoid division by zero at r=0
# np.divide(..., where=R != 0) -> only divide where R != 0, otherwise use 0
a = np.divide(X, R, out=np.zeros_like(X), where=R != 0)
b = np.divide(Y, R, out=np.zeros_like(Y), where=R != 0)

# -----------------------------
# Initial condition u(x,y,0)
# -----------------------------

# Allocate solution array at current time step, shape (Ny, Nx)
# (row index corresponds to y, column index to x)
u = np.zeros((Ny, Nx))

# Initial "blob": u = 1 inside a circle of radius 12.5 centered at (0,0),
#                 u = 0 elsewhere
u[(X**2 + Y**2) < (12.5)**2] = 1.0

# List to store solution snapshots for visualization
frames = []

# -----------------------------
# Time-stepping loop
# -----------------------------
for n in range(Nt):
    # Copy current solution to u_new, which will store u^{n+1}
    u_new = u.copy()

    # Arrays for first and second derivatives
    u_x = np.zeros_like(u)    # first derivative ∂u/∂x
    u_y = np.zeros_like(u)    # first derivative ∂u/∂y
    u_xx = np.zeros_like(u)   # second derivative ∂²u/∂x²
    u_yy = np.zeros_like(u)   # second derivative ∂²u/∂y²

    # -----------------------------
    # Spatial derivatives (finite differences)
    # -----------------------------

    # Upwind scheme in x-direction:
    #   u_x ≈ (u[i,j] - u[i,j-1]) / dx   for interior points j = 1..Nx-2
    #   Left neighbor is used → "upwind" with respect to positive x-direction.
    u_x[:, 1:-1] = (u[:, 1:-1] - u[:, :-2]) / dx

    # Upwind scheme in y-direction:
    #   u_y ≈ (u[i,j] - u[i-1,j]) / dy   for interior points i = 1..Ny-2
    u_y[1:-1, :] = (u[1:-1, :] - u[:-2, :]) / dy

    # Second derivative in x (central difference):
    #   u_xx ≈ (u[i,j+1] - 2u[i,j] + u[i,j-1]) / dx²
    u_xx[:, 1:-1] = (u[:, 2:] - 2 * u[:, 1:-1] + u[:, :-2]) / dx**2

    # Second derivative in y (central difference):
    #   u_yy ≈ (u[i+1,j] - 2u[i,j] + u[i-1,j]) / dy²
    u_yy[1:-1, :] = (u[2:, :] - 2 * u[1:-1, :] + u[:-2, :]) / dy**2

    # -----------------------------
    # Explicit time update (Euler forward)
    # -----------------------------
    # For interior nodes (excluding boundaries), update using:
    #
    #   u^{n+1} = u^{n}
    #             + dt * [ -a u_x - b u_y + u_xx + u_yy ]
    #
    # which corresponds to:
    #   u_t + a u_x + b u_y = u_xx + u_yy
    #
    u_new[1:-1, 1:-1] += dt * (
        -a[1:-1, 1:-1] * u_x[1:-1, 1:-1]
        -b[1:-1, 1:-1] * u_y[1:-1, 1:-1]
        + u_xx[1:-1, 1:-1]
        + u_yy[1:-1, 1:-1]
    )

    # -----------------------------
    # Neumann boundary conditions (zero normal derivative)
    # -----------------------------
    # Neumann BC ∂u/∂n = 0 is enforced by copying interior neighbor values:
    #
    #   left boundary   (x = x_min):  u[:, 0]  = u[:, 1]
    #   right boundary  (x = x_max):  u[:, -1] = u[:, -2]
    #   bottom boundary (y = y_min):  u[0, :]  = u[1, :]
    #   top boundary    (y = y_max):  u[-1,:]  = u[-2,:]
    #
    u_new[:, 0] = u_new[:, 1]
    u_new[:, -1] = u_new[:, -2]
    u_new[0, :] = u_new[1, :]
    u_new[-1, :] = u_new[-2, :]

    # Update solution for next time step
    u = u_new.copy()

    # Store every 5th time step for animation (coarser temporal sampling)
    if n % 5 == 0:
        frames.append(u.copy())


# -----------------------------
# Visualization: contour animation
# -----------------------------
fig, ax = plt.subplots(figsize=(6, 5))

# Initial contour plot at the first stored frame
contour = ax.contourf(X, Y, frames[0], levels=50, cmap='plasma')
fig.colorbar(contour)

def update(frame_idx):
    """
    Update function for the animation.

    Args:
        frame_idx: index into 'frames' list (0,1,2,...)

    Plots u(x,y,t) at the corresponding time and updates the title.
    """
    ax.clear()
    # Each stored frame corresponds to 5 time steps → effective time = frame_idx * dt * 5
    ax.set_title(f"Time = {frame_idx * dt * 5:.2f} s")
    ax.contourf(X, Y, frames[frame_idx], levels=50, cmap='plasma')
    return []

# Build the animation: frames=len(frames) and 100 ms between frames
ani = animation.FuncAnimation(fig, update, frames=len(frames), interval=100)

# Convert animation to JS/HTML for display in a Jupyter notebook
HTML(ani.to_jshtml())


In [ ]:
# === Match PINN and FDM frames (aligning t values) ===

frames_pinn = []  # list to store PINN snapshots u_PINN(x,y,t) for each time

# t-values used by the FDM animation (same number of frames)
# We want PINN predictions at exactly the same times for fair comparison.
t_vals = np.linspace(0, 10, len(frames))  # frames = list of FDM snapshots

for tval in t_vals:
    # Build a grid of (x, y, t) points for the current time tval:
    #   - X.ravel(), Y.ravel() give flattened spatial coordinates
    #   - np.full_like(X.ravel(), tval) gives a matching-length array with constant tval
    xyt = np.stack(
        [X.ravel(), Y.ravel(), np.full_like(X.ravel(), tval)],
        axis=1
    )  # shape: (Nx*Ny, 3)

    # Convert to a PyTorch tensor and move to the chosen device (CPU/GPU)
    xyt_tensor = torch.tensor(xyt, dtype=torch.float32).to(device)

    # Predict u(x,y,tval) using the trained PINN without tracking gradients
    with torch.no_grad():
        # model output is (Nx*Ny, 1); reshape to (Ny, Nx) to match FDM grid layout
        u_pred = model(xyt_tensor).cpu().numpy().reshape(X.shape)

    # Store this time slice in the list of PINN frames
    frames_pinn.append(u_pred)


# === 1. Animated Side-by-Side Contour Plot ===

# Create a figure with two subplots side-by-side:
#   ax1: FDM solution
#   ax2: PINN solution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Initial contour plots for time t = t_vals[0]
c1 = ax1.contourf(X, Y, frames[0], cmap='plasma', levels=50)
c2 = ax2.contourf(X, Y, frames_pinn[0], cmap='plasma', levels=50)

# Separate colorbars for each subplot
fig.colorbar(c1, ax=ax1)
fig.colorbar(c2, ax=ax2)

def update(frame_idx):
    """
    Update function for the side-by-side FDM vs PINN animation.

    Args:
        frame_idx: index of the current frame (0..len(frames)-1)

    Behavior:
        - Clears both axes.
        - Sets titles indicating the method and current time.
        - Draws contour plots of FDM and PINN solutions at that time.
    """
    # Clear previous contours
    ax1.clear()
    ax2.clear()

    # Titles show which method and the corresponding time value
    ax1.set_title(f"FDM t={t_vals[frame_idx]:.2f}")
    ax2.set_title(f"PINN t={t_vals[frame_idx]:.2f}")

    # Axis labels for both plots
    ax1.set_xlabel("x")
    ax1.set_ylabel("y")
    ax2.set_xlabel("x")
    ax2.set_ylabel("y")

    # Draw updated filled contour plots for FDM (left) and PINN (right)
    ax1.contourf(X, Y, frames[frame_idx], cmap='plasma', levels=50)
    ax2.contourf(X, Y, frames_pinn[frame_idx], cmap='plasma', levels=50)
    
    return []  # required by FuncAnimation, even if we don't return artists

# Create the animation over all stored frames
# interval=100 → 100 ms between frames (≈10 fps)
ani = animation.FuncAnimation(fig, update, frames=len(frames), interval=100)

# Save the animation as a GIF file using the Pillow writer
ani.save("comparison.gif", writer="pillow", fps=10)

# Display the animation inline in a Jupyter notebook
HTML(ani.to_jshtml())


In [ ]:
snap_times = [0, 2.5, 5.0, 7.5, 10.0]
snap_indices = [np.argmin(np.abs(t_vals - t)) for t in snap_times]

for idx, t in zip(snap_indices, snap_times):
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].set_title(f"FDM at t={t:.1f}")
    axs[1].set_title(f"PINN at t={t:.1f}")
    ax1.set_xlabel("x")
    ax1.set_ylabel("y")
    ax2.set_xlabel("x")
    ax2.set_ylabel("y")
    axs[0].contourf(X, Y, frames[idx], cmap='plasma', levels=50)
    axs[1].contourf(X, Y, frames_pinn[idx], cmap='plasma', levels=50)
    plt.tight_layout()
    plt.show()

In [ ]:
y0_index = np.argmin(np.abs(y))  # index where y ≈ 0

for idx, t in zip(snap_indices, snap_times):
    u_fdm_line = frames[idx][y0_index, :]
    u_pinn_line = frames_pinn[idx][y0_index, :]
    plt.figure(figsize=(6,4))
    plt.plot(x, u_fdm_line, label='FDM', linewidth=2)
    plt.plot(x, u_pinn_line, '--', label='PINN', linewidth=2)
    plt.title(f"Centerline at y=0, t={t:.1f}")
    plt.xlabel("x"); plt.ylabel("u(x,0,t)")
    plt.legend(); plt.grid(True)
    plt.show()